# Exploring Gender Bias in Word Embeddings

This lab is based on a tutorial for the responsibly Python package. You can view the original tutorial and learn more about responsibly here: [https://learn.responsibly.ai/word-embedding](https://learn.responsibly.ai/word-embedding).

Please go through this notebook by running each cell and answering the questions which occur at the end of each part. Some will involve you coding while others will simply require that you write a response in markdown. The questions are marked by whales (🐳).

#### Note: You will be using a smaller version of the the full Word2Vec (which compressed is still about 2GB). You may find that some words you want to use for the assignment are not included in the smaller dictionary. You may have to find words to use by trial and error.

# Part 1: Setup

## 1.1 - Install and load packages

- On a mac, open a terminal window
- make a conda environment with strictly less than Python 3.8 but greater than or equal to Python 3.6

conda create -n py37 python=3.7 anaconda

conda activate py37

- **Install numpy and matplotlib (v2.2.3) through conda**

conda install numpy

conda install matplotlib=2.2.3

pip install responsibly

- On a mac you then load Jupyter Notebook by typing the command below.  The ampersand is to run the notebook in the "background", that is to allow you to continue typing other commands in the same terminal window.

jupyter-notebook& 



In [ ]:
from IPython.display import Image
import warnings
from math import acos, degrees
from operator import itemgetter
from copy import deepcopy

# matplotlib
from matplotlib import pylab as plt
import matplotlib.pylab as plt

# numpy
from numpy.testing import assert_almost_equal
from numpy.linalg import norm

# scikit-learn
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer

# responsibly
import responsibly
from responsibly.we import most_similar
from responsibly.we import GenderBiasWE
from responsibly.we import load_w2v_small
from responsibly.we import calc_all_weat
from responsibly.we import calc_single_weat
from responsibly.we import GenderBiasWE


## 1.2 - Validate Installation of `responsibly`

In [ ]:
# You should get '0.1.3'
responsibly.__version__

# Part 2: Motivation - Why use word embeddings?

## 2.1 - [NLP (Natural Language Processing)](https://en.wikipedia.org/wiki/Natural_language_processing)

Partial list of tasks and applications:

- Classification
- Machine Translation
- Information Retrieval
- Conversation Chatbots
- Coreference Resolution

In [ ]:
Image("./images/corefexample.png", width=600)


<small>Source: [Stanford Natural Language Processing Group](https://nlp.stanford.edu/projects/coref.shtml)</small>

## 2.2 - Machine Learning (NLP) Pipeline

In [ ]:
Image("./images/nlp-pipeline.png", width=600)


<small>Source: [Kai-Wei Chang (UCLA) - What It Takes to Control Societal Bias in Natural Language Processing](https://www.youtube.com/watch?v=RgcXD_1Cu18)</small>

## 2.3 - How can we represent language to a machine?

We need some kind of *dictionary* to transform/encode from a human representation to a machine representation (words → numbers)

### Bag of Words (for a text)


In [ ]:
Image("./images/bow.png", width=300)

<small>Source: Zheng, A.& Casari, A. (2018). Feature Engineering for Machine Learning. O'Reilly Media.</small>

In [ ]:
vocabulary = ['it', 'they', 'puppy', 'and', 'cat', 'aardvark', 'cute', 'extremely', 'not']

vectorizer = CountVectorizer(vocabulary=vocabulary)

In [ ]:
sentence = 'it is a puppy and it is extremely cute'

In [ ]:
vectorizer.fit_transform([sentence]).toarray()

In [ ]:
vectorizer.fit_transform(['it is not a puppy and it is extremely cute']).toarray()

In [ ]:
vectorizer.fit_transform(['it is a puppy and it is extremely not cute']).toarray()

Read more about scikit-learn's text feature extraction [here](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).

### One-hot representation

In [ ]:
[vectorizer.fit_transform([word]).toarray()
 for word in sentence.split()
 if word in vocabulary]

### A problem with one-hot representation


In [ ]:
Image("./images/audio-image-text.png", width=600)

<small> Source: [Tensorflow Documentation](https://www.tensorflow.org/tutorials/representation/word2vec) </small>

### Embedding a word in a n-dimensional space

Distance ~ Semantic Similarity

Again, for the purposes of this assignment, these are just references and additional sources of information. You should have already seen "The Illustrated Word2vec" by Jay Alammar.

#### Examples (algorithms and pre-trained models)
- [Word2Vec](https://code.google.com/archive/p/word2vec/)
- [GloVe](https://nlp.stanford.edu/projects/glove/)
- [fastText](https://fasttext.cc/)
- [ELMo](https://allennlp.org/elmo) (contextualized)

#### Training: using *word-context* relationships from a corpus.

See: [The Illustrated Word2vec by Jay Alammar](http://jalammar.github.io/illustrated-word2vec/)

#### State of the Art - Contextual Word Embedding → Language Models
- [The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning) by Jay Alammar](http://jalammar.github.io/illustrated-bert/)
- Microsoft - [NLP Best Practices](https://github.com/microsoft/nlp-recipes)
- [Tracking Progress in Natural Language Processing](https://nlpprogress.com/)

## 🐳 **Questions** 🐳

- What is the difference between the bag-of-words approach and the one-hot encoding demonstrated here?


+ Can you think of a use case where one-hot encoding or a bag of words approach would be better than a high-dimensional word embedding? If so, why? If not, why not?

# Part 3: Playing with Word2Vec word embeddings


[Word2Vec](https://code.google.com/archive/p/word2vec/) - Google News - 100B tokens, 3M vocab, cased, 300d vectors - only lowercase vocab extracted

Loaded using [`responsibly`](http://docs.responsibly.ai) package, the function [`responsibly.we.load_w2v_small`]() returns a [`gensim`](https://radimrehurek.com/gensim/)'s [`KeyedVectors`](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors) object.

## 3.1 - Basic Properties

In [ ]:
# ignore warnings
# generally, you shouldn't do that, but for this tutorial we'll do so for the sake of simplicity
warnings.filterwarnings('ignore')

In [ ]:
# this loads a small version of Word2Vec which does not include the full vocabulary 
w2v_small = load_w2v_small()

In [ ]:
# vocabulary size
len(w2v_small.vocab)

In [ ]:
# get the vector of the word "home"
print('home =', w2v_small['home'])

In [ ]:
# the word embedding dimension, in this case, is 300

len(w2v_small['home'])

In [ ]:
# all the words are normalized (=have norm equal to one as vectors)

norm(w2v_small['home'])

In [ ]:
# make sure that all the vectors are normalized!

length_vectors = norm(w2v_small.vectors, axis=1)

assert_almost_equal(actual=length_vectors,
                    desired=1,
                    decimal=5)

## 3.2 - Mesuring Distance between Words

### Mesure of Similiarty: [Cosine Similariy](https://en.wikipedia.org/wiki/Cosine_similarity)
- Measures the cosine of the angle between two vecotrs.
- Ranges between 1 (same vector) to -1 (opposite/antipode vector)
- In Python, for normalized vectors (Numpy's array), use the `@`(at) operator

In [ ]:
w2v_small['cat'] @ w2v_small['cat']

In [ ]:
w2v_small['cat'] @ w2v_small['cats']

In [ ]:
degrees(acos(w2v_small['cat'] @ w2v_small['cats']))

In [ ]:
w2v_small['cat'] @ w2v_small['dog']

In [ ]:
degrees(acos(w2v_small['cat'] @ w2v_small['dog']))

In [ ]:
w2v_small['cat'] @ w2v_small['cow']

In [ ]:
degrees(acos(w2v_small['cat'] @ w2v_small['cow']))

In [ ]:
w2v_small['cat'] @ w2v_small['graduated']

In [ ]:
degrees(acos(w2v_small['cat'] @ w2v_small['graduated']))

In general, the use of word embeddings to encode words, as an input for NLP systems*, improves their performance compared to one-hot representation.

\* Sometimes the embedding is learned as part of the NLP system.

## 3.3 - Visualizing Word Embedding in 2D using T-SNE

<small>Source: [Google's Seedbank](https://research.google.com/seedbank/seed/pretrained_word_embeddings)</small>

In [ ]:


# take the most common words in the corpus between 200 and 600
words = [word for word in w2v_small.index2word[200:600]]

# convert the words to vectors
embeddings = [w2v_small[word] for word in words]

# perform T-SNE
words_embedded = TSNE(n_components=2).fit_transform(embeddings)

# ... and visualize!
plt.figure(figsize=(20, 20))
for i, label in enumerate(words):
    x, y = words_embedded[i, :]
    plt.scatter(x, y)
    plt.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                 ha='right', va='bottom', size=11)
plt.show()

#### Extra: [Tensorflow Embedding Projector](http://projector.tensorflow.org)

## 3.4 - Most Similar

What are the most simlar words to a given word?

In [ ]:
w2v_small.most_similar('cat')

Given a list of words, which one doesn't match?

(The word furthest away from the mean of all words.)

In [ ]:
w2v_small.doesnt_match('breakfast cereal dinner lunch'.split())

## 3.5 - Vector Arithmetic

In [ ]:
Image("./images/vector-addition.png", width="400")


<small>Source: [Wikipedia](https://commons.wikimedia.org/wiki/File:Vector_add_scale.svg)</small>

In [ ]:
# nature + science = ?

w2v_small.most_similar(positive=['nature', 'science'])

## 3.6 - Vector Analogy

In [ ]:
Image("./images/linear-relationships.png", width=800)

<small>Source: [Tensorflow Documentation](https://www.tensorflow.org/tutorials/representation/word2vec)</small>

We now try to represent the following analogy: 

*king is to man as ____ is to woman*

We can think of this as $\overrightarrow{king} - \overrightarrow{man} + \overrightarrow{woman}$

In [ ]:
w2v_small.most_similar(positive=['king', 'woman'],
                       negative=['man'])

## 3.7 - Think about a direction in word embedding as a relation

Remember: direction is not a word vector by itself

$\overrightarrow{big} - \overrightarrow{small} + \overrightarrow{smaller}$

In [ ]:
w2v_small.most_similar(positive=['big', 'smaller'],
                       negative=['small'])

Sometimes vector arithmetic does not behave as expect. Notice how close the top two results are in the output that follows.

$\overrightarrow{up} - \overrightarrow{down} + \overrightarrow{backward}$

(we include backwards in the code, otherwise it will be the top result)

In [ ]:
w2v_small.most_similar(positive=['up', 'backward', 'backwards'],
                       negative=['down'])

#### Extra Information

Keep in mind the word embedding was generated by learning the co-occurrence of words, so the fact that it *empirically* exhibits "concept arithmetic", it doesn't necessarily mean it learned it! In fact, it seems it didn't.
See: [king - man + woman is queen; but why? by Piotr Migdał](https://p.migdal.pl/2017/01/06/king-man-woman-queen-why.html)

[Demo - Word Analogies Visualizer by Julia Bazińska](https://lamyiowce.github.io/word2viz/)

In fact, `w2v_small.most_similar` finds the closest word which *is not one* of the given ones. This is a real methodological issue. Nowadays, it is not a common practice anymore to evaluate word embedding with analogies.

You can use [`responsibly.we.most_similar`](https://docs.responsibly.ai/word-embedding-bias.html#responsibly.we.utils.most_similar) for the unrestricted version.

## 🐳 **Questions** 🐳

+ What are some of the advantages and disadvantages of the 2-dimensional visualization in **3.3**?

<br>

+ What makes sense to you about what you see in the 2-dimensional visualization in **3.3** and what doesn't?

<br>

+ Why did the vector arithmetic in **3.6** appear to work but it did not in the latter part of **3.7**?

<br>

+ Find the words that are most similar to this relation: *see* is to *eye* as ___ is to *ear*

In [ ]:
### Your code here

+ Pick a relation of your own and find words most similar to it. What did you expect to see, and what did you find? (Note that the dictionary used here is limited and you may have to experiment to find words that are included.)

In [ ]:
### Your code here

# Part 4: Gender Bias


Remember that Word2Vec is primarily trained on Google News.

Bolukbasi Tolga, Kai-Wei Chang, James Y. Zou, Venkatesh Saligrama, and Adam T. Kalai. [Man is to computer programmer as woman is to homemaker? debiasing word embeddings](https://arxiv.org/abs/1607.06520). NIPS 2016.

In [ ]:
from responsibly.we import load_w2v_small

w2v_small = load_w2v_small()

The method of generating analogies can enforce producing gender sterotyping ones -

Nissim, M., van Noord, R., van der Goot, R. (2019). [Fair is Better than Sensational: Man is to Doctor as Woman is to Doctor](https://arxiv.org/abs/1905.09866).

... and a [Twitter thread](https://twitter.com/adamfungi/status/1133865428663635968) between the authors of the two papers.

## 4.1 - What can we take from analogies?

Can we describe some sort of "gender direction"?

$\overrightarrow{she} - \overrightarrow{he}$

Notice the order - words in the "she" direction will be positive, while words in the "he" direction will be negative.

In [ ]:
gender_direction = w2v_small['she'] - w2v_small['he']

gender_direction /= norm(gender_direction)

In practice, we calculate the gender direction using multiple definitional pair of words for better estimation (words may have more than one meaning):

- woman - man
- girl - boy
- she - he
- mother - father
- daughter - son
- gal - guy
- female - male
- her - his
- herself - himself
- Mary - John

## 4.2 - Projections

In [ ]:
gender_direction @ w2v_small['architect']

In [ ]:
gender_direction @ w2v_small['interior_designer']

The word *architect* appears in more contexts with *he* than with *she*, and vice versa for *interior designer*.

In [ ]:
w2v_small_gender_bias = GenderBiasWE(w2v_small, only_lower=True)

In [ ]:
w2v_small_gender_bias.positive_end, w2v_small_gender_bias.negative_end

In [ ]:
# gender direction
w2v_small_gender_bias.direction[:10]

In [ ]:
from responsibly.we.data import BOLUKBASI_DATA

neutral_profession_names = BOLUKBASI_DATA['gender']['neutral_profession_names']

In [ ]:
neutral_profession_names[:10]

In [ ]:
len(neutral_profession_names)

In [ ]:
# this is the same as using the @ operator on the bias direction
w2v_small_gender_bias.project_on_direction(neutral_profession_names[0])

**Let's visualize the projections of professions (neutral and specific by the orthography) on the gender direction**

Note that we visuzalize the most extreme professions

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))

w2v_small_gender_bias.plot_projection_scores(n_extreme=20, ax=ax);

**Extra** - Visualizing gender bias with [Word Clouds](http://wordbias.umiacs.umd.edu/)

This is an example of the type of work that you could do for a project

## 4.3 - Are the projections of occupation words on the gender direction related to the real world?

Let's take the percentage of female in various occupations from the Labor Force Statistics of 2017 Population Survey.

Taken from: [https://arxiv.org/abs/1804.06876](https://arxiv.org/abs/1804.06876)

In [ ]:
from responsibly.we.data import OCCUPATION_FEMALE_PRECENTAGE

sorted(OCCUPATION_FEMALE_PRECENTAGE.items(), key=itemgetter(1))

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 8))

w2v_small_gender_bias.plot_factual_association(ax=ax);

## 4.4 Word Embeddings Encoding Gender Stereotypes

Garg, N., Schiebinger, L., Jurafsky, D., & Zou, J. (2018). [Word embeddings quantify 100 years of gender and ethnic stereotypes](https://www.pnas.org/content/pnas/115/16/E3635.full.pdf). Proceedings of the National Academy of Sciences, 115(16), E3635-E3644.

In [ ]:
Image("./images/gender-bias-over-decades.png")

<small>Data: Google Books/Corpus of Historical American English (COHA)</small>

## 4.5 - Direct Bias Measure

1. Project each **neutral profession name** on the gender direction
2. Calculate the absolute value of each projection
3. Average it all

In [ ]:
# using responsibly
w2v_small_gender_bias.calc_direct_bias()

In [ ]:
# what responsibly does:
neutral_profession_projections = [w2v_small[word] @ w2v_small_gender_bias.direction
                                  for word in neutral_profession_names]

abs_neutral_profession_projections = [abs(proj) for proj in neutral_profession_projections]

sum(abs_neutral_profession_projections) / len(abs_neutral_profession_projections)

## 4.6 - Indirect Bias Measure
Similarity due to shared "gender direction" projection

In [ ]:
w2v_small_gender_bias.generate_closest_words_indirect_bias('softball',
                                                           'football')

## 🐳 **Questions** 🐳

- Building off the example in section **4.2**, calculate three additional projects of the gender direction onto three words of your choice

In [ ]:
# Calculate three additional gender projections here


- What do you think of the idea of a gender direction as quantified in section **4.1** above?

    - What do you think the formula does right?

    - What do you think the formula does wrong?

- Using the methods shown in **4.1** and **4.2**, define a new direction for a concept of your choice besides gender

    - What is your concept?
    
    - What are your starting and ending words?

- Calculate the new concept direction, and project it onto three words of your choice below

In [ ]:
### Calculate the new direction


### Project it onto three words of your choice

- What do your results tell you?

# Part 5: Mitigating Bias

> We intentionally do not reference the resulting embeddings as "debiased" or free from all gender bias, and
prefer the term "mitigating bias" rather that "debiasing," to guard against the misconception that the resulting
embeddings are entirely "safe" and need not be critically evaluated for bias in downstream tasks. <small>James-Sorenson, H., & Alvarez-Melis, D. (2019). [Probabilistic Bias Mitigation in Word Embeddings](https://arxiv.org/pdf/1910.14497.pdf). arXiv preprint arXiv:1910.14497.</small>

## 5.1 - Neutralize

We will remove the gender projection from all the words, except the gender-specific ones, and then normalize.

We need to "learn" what are the gender-specific words in the vocabulary for a seed set of gender-specific words (we do this by semi-automatic use of [WordNet](https://en.wikipedia.org/wiki/WordNet))

In [ ]:
w2v_small_gender_debias = w2v_small_gender_bias.debias(method='neutralize', inplace=False)

In [ ]:
print('home:',
      'before =', w2v_small_gender_bias.model['home'] @ w2v_small_gender_bias.direction,
      'after = ', w2v_small_gender_debias.model['home'] @ w2v_small_gender_debias.direction)

In [ ]:
print('man:',
      'before =', w2v_small_gender_bias.model['man'] @ w2v_small_gender_bias.direction,
      'after = ', w2v_small_gender_debias.model['man'] @ w2v_small_gender_debias.direction)

In [ ]:
print('woman:',
      'before =', w2v_small_gender_bias.model['woman'] @ w2v_small_gender_bias.direction,
      'after = ', w2v_small_gender_debias.model['woman'] @ w2v_small_gender_debias.direction)

In [ ]:
w2v_small_gender_debias.calc_direct_bias()

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))

w2v_small_gender_debias.plot_projection_scores(n_extreme=20, ax=ax);

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 8))

w2v_small_gender_debias.plot_factual_association(ax=ax);

## 5.2 Equalize

In [ ]:
w2v_small_gender_debias.model['grandfather'] @ w2v_small_gender_debias.model['babysitter']

In [ ]:
w2v_small_gender_debias.model['grandmother'] @ w2v_small_gender_debias.model['babysitter']

Notice that the projection of "grandfather" and "grandmother" onto the now neutral word "babysitter" are not equal, even though we have "debiased" our embedding. We might prefer that in a "debiased" embedding, "grandfather" and "grandmother" have roughly the same projection onto neutral words. Below you can see an example of some of the pairs we are interested in equalizing.

In [ ]:
BOLUKBASI_DATA['gender']['equalize_pairs'][:10]

## 5.3 Hard Debias = Neutralize + Equalize

Now we debias by both neutralizing as we did before along with equalizing across the pairs we indentified above (note that the list that was printed out is not exhaustive).

In [ ]:
w2v_small_gender_debias = w2v_small_gender_bias.debias(method='hard', inplace=False)

In [ ]:
print('home:',
      'before =', w2v_small_gender_bias.model['home'] @ w2v_small_gender_bias.direction,
      'after = ', w2v_small_gender_debias.model['home'] @ w2v_small_gender_debias.direction)

In [ ]:
print('man:',
      'before =', w2v_small_gender_bias.model['man'] @ w2v_small_gender_bias.direction,
      'after = ', w2v_small_gender_debias.model['man'] @ w2v_small_gender_debias.direction)

In [ ]:
print('woman:',
      'before =', w2v_small_gender_bias.model['woman'] @ w2v_small_gender_bias.direction,
      'after = ', w2v_small_gender_debias.model['woman'] @ w2v_small_gender_debias.direction)

In [ ]:
w2v_small_gender_debias.calc_direct_bias()

In [ ]:
w2v_small_gender_debias.model['grandfather'] @ w2v_small_gender_debias.model['babysitter']

In [ ]:
w2v_small_gender_debias.model['grandmother'] @ w2v_small_gender_debias.model['babysitter']

Notice that "grandfather" and "grandmother" now have (almost exactly) the same projection onto "babysitter".

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))

w2v_small_gender_debias.plot_projection_scores(n_extreme=20, ax=ax);

The disadvantage of equalization is that it might remove meaningful associations, such as the verb meaning of "grandfather", e.g. "to grandfather a regulation". Equalization removes this distinction.

## 5.4 - Compare Preformances

After debiasing, the performance of the word embedding (using standard benchmarks) only gets slightly worse!

In [ ]:
w2v_small_gender_bias.evaluate_word_embedding()

In [ ]:
w2v_small_gender_debias.evaluate_word_embedding()

## 🐳 **Questions** 🐳

- What is "neutralizing", and what is "equalizing" in the context of "hard debiasing", and how do they differ? 
    
    <small>You may find it helpful to look at the following paper from class (you may copy the definitions from the paper, but the explanation on how they differ should be your own):</small>
    <small>[Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings](https://arxiv.org/pdf/1607.06520.pdf)</small>

    